Ashima Fatima Seik Mugibur Raghman, 21BAI1830

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install nltk scikit-learn pandas

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import zipfile
import os
import re
import string

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
zip_file_path = '/content/drive/MyDrive/SEM7_SLP/sentiment+labelled+sentences.zip'

with zipfile.ZipFile(zip_file_path, 'r') as z:
    z.extractall('/content/extracted_data')

extracted_files = os.listdir('/content/extracted_data')
print("Extracted files:", extracted_files)

Extracted files: ['__MACOSX', 'sentiment labelled sentences']


In [8]:
sentence_dir = '/content/extracted_data/sentiment labelled sentences'

In [ ]:
'''file_path = '/content/extracted_data/sentiment labelled sentences/yelp_labelled.txt'  # Replace with your file path

with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()
    print(content)'''

In [12]:
data = []
for annot_file in os.listdir(sentence_dir):
    file_path = os.path.join(sentence_dir, annot_file)
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        sentences = file.readlines()
        for sentence in sentences:
            parts = sentence.strip().rsplit('\t', 1)
            if len(parts) == 2:  # Ensure there are exactly two parts: sentence and label
                sentence_text, sentiment_label = parts
                data.append([sentence_text, sentiment_label])

In [13]:
df = pd.DataFrame(data, columns=['sentence', 'sentiment'])
df.head()

,sentence,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


# Text Classification

In [22]:
def txt_classification(text):
    X_train, X_test, y_train, y_test = train_test_split(df[text], df['sentiment'], test_size=0.3, random_state=42)

    tfidf_vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
    X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

    nb_classifier = MultinomialNB()
    nb_classifier.fit(X_train_tfidf, y_train)
    y_pred = nb_classifier.predict(X_test_tfidf)

    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# With Preprocessing

In [16]:
#initialize NLTK tools
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [17]:
def preprocess_text(text, remove_stopwords=True, use_stemming=False, use_lemmatization=False):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    if remove_stopwords:
        tokens = [word for word in tokens if word not in stopwords.words('english')]
    if use_stemming:
        tokens = [stemmer.stem(word) for word in tokens]
    elif use_lemmatization:
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Rejoin tokens into a single string
    return ' '.join(tokens)

In [18]:
df['preprocessed_text'] = df['sentence'].apply(lambda x: preprocess_text(x, remove_stopwords=True))

In [19]:
df['preprocessed_text_stemmed'] = df['sentence'].apply(lambda x: preprocess_text(x, remove_stopwords=True, use_stemming=True))
df['preprocessed_text_lemmatized'] = df['sentence'].apply(lambda x: preprocess_text(x, remove_stopwords=True, use_lemmatization=True))

In [20]:
df.head()

,sentence,sentiment,preprocessed_text,preprocessed_text_stemmed,preprocessed_text_lemmatized
0,Wow... Loved this place.,1,wow loved place,wow love place,wow loved place
1,Crust is not good.,0,crust good,crust good,crust good
2,Not tasty and the texture was just nasty.,0,tasty texture nasty,tasti textur nasti,tasty texture nasty
3,Stopped by during the late May bank holiday of...,1,stopped late may bank holiday rick steve recom...,stop late may bank holiday rick steve recommen...,stopped late may bank holiday rick steve recom...
4,The selection on the menu was great and so wer...,1,selection menu great prices,select menu great price,selection menu great price


# Results

In [23]:
txt_classification('sentence')

Accuracy: 0.8055555555555556
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.81      0.81       465
           1       0.80      0.80      0.80       435

    accuracy                           0.81       900
   macro avg       0.81      0.81      0.81       900
weighted avg       0.81      0.81      0.81       900

Confusion Matrix:
 [[376  89]
 [ 86 349]]


In [24]:
txt_classification('preprocessed_text')

Accuracy: 0.7933333333333333
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.75      0.79       465
           1       0.76      0.83      0.80       435

    accuracy                           0.79       900
   macro avg       0.80      0.79      0.79       900
weighted avg       0.80      0.79      0.79       900

Confusion Matrix:
 [[351 114]
 [ 72 363]]


In [25]:
txt_classification('preprocessed_text_stemmed')

Accuracy: 0.7922222222222223
Classification Report:
               precision    recall  f1-score   support

           0       0.82      0.77      0.79       465
           1       0.77      0.81      0.79       435

    accuracy                           0.79       900
   macro avg       0.79      0.79      0.79       900
weighted avg       0.79      0.79      0.79       900

Confusion Matrix:
 [[359 106]
 [ 81 354]]


In [26]:
txt_classification('preprocessed_text_lemmatized')

Accuracy: 0.7855555555555556
Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.74      0.78       465
           1       0.75      0.83      0.79       435

    accuracy                           0.79       900
   macro avg       0.79      0.79      0.79       900
weighted avg       0.79      0.79      0.79       900

Confusion Matrix:
 [[344 121]
 [ 72 363]]


# Difference in texts to explain the results

In [27]:
for i in range(1, 6):
    print("Original:", df['sentence'].iloc[i])
    print("Preprocessed:", df['preprocessed_text'].iloc[i])
    print("\n")

Original: Crust is not good.
Preprocessed: crust good


Original: Not tasty and the texture was just nasty.
Preprocessed: tasty texture nasty


Original: Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.
Preprocessed: stopped late may bank holiday rick steve recommendation loved


Original: The selection on the menu was great and so were the prices.
Preprocessed: selection menu great prices


Original: Now I am getting angry and I want my damn pho.
Preprocessed: getting angry want damn pho


